In [ ]:
import os

from ultralytics import YOLO
from ultralytics import settings

from ray import train, tune
from ray.tune import ResultGrid

import argparse
import sys

In [ ]:
def get_args_parse():
    parser = argparse.ArgumentParser("Exploratory Data Analysis")
    parser.add_argument("--data", default="/hpc/home/csr33/ast_object_detection/ast.yaml", type=str)
    parser.add_argument("--model", default='yolov8n.pt', type=str)
    args = parser.parse_args()
    return args
sys.argv = ['my_notebook']
args = get_args_parse()

In [ ]:
model = YOLO('yolov8n.pt')
model.tune(data=args.data, epochs=5, iterations=2)

In [ ]:
#description of model metrics
https://github.com/ultralytics/ultralytics/issues/2789#issuecomment-1808018730
/tmp/tmp_vsfwz0p/test_tune1/weights/best.pt...


In [4]:
os.chdir("/work/csr33/object_detection")

In [5]:
# Update settings
# https://docs.ultralytics.com/quickstart/
os.makedirs('/work/csr33/object_detection/weights', exist_ok=True)
settings.update({'weights_dir': '/work/csr33/object_detection/weights'})

os.makedirs('/work/csr33/object_detection/runs', exist_ok=True)
settings.update({'runs_dir': '/work/csr33/object_detection/runs'})

settings.update({'neptune': False, 'clearml': False, 
                 "raytune": True, #'comet': True, 'raytune': True,
                 'dvc': False, 'hub': False, 'mlflow': False,
                 'tensorboard': False, 'wandb': False})

# Reset settings to default values #settings.reset()

In [6]:
import torch

torch.cuda.set_device(0) # Set to your desired GPU number

In [8]:
 trainable="_tune"

In [9]:
# Load a YOLOv8n model
model = YOLO(args.model) # load a pretrained model (recommended for training)

In [10]:
exp_name = "test_tune4" #"tune_analyzing_results"

In [12]:
# https://docs.ray.io/en/latest/tune/examples/tune_analyze_results.html
# https://docs.ray.io/en/latest/tune/index.html
# https://docs.ultralytics.com/reference/engine/tuner/?h=tune
# Tune hyperparameters on COCO8 for 30 epochs
# Use model.tune to tune the hyperparameters

# Start tuning hyperparameters for YOLOv8n training on the COCO8 dataset

#parallel on single gpu
#https://discuss.ray.io/t/how-do-i-run-my-experiment-on-a-single-gpu/9689


result_grid = model.tune(data=args.data, epochs=1, iterations=2,
                         gpu_per_trial = 0.5, #https://github.com/ultralytics/ultralytics/issues/4133
                         #cpu_per_trial =2,
                         workers=1, 
                         project="/work/csr33/object_detection",
                         name=exp_name,
                         use_ray=True, grace_period = 1) #grace period must be less than the number of epochs

2024-01-30 14:48:03,715	INFO wandb.py:307 -- Already logged into W&B.
2024-01-30 14:48:36,391	ERROR tune_controller.py:1374 -- Trial task failed for trial _tune_7b2c1_00001
Traceback (most recent call last):
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/_private/worker.py", line 2624, in get
    raise value.as_instan

(_tune pid=1928629) New https://pypi.org/project/ultralytics/8.1.8 available 😃 Update with 'pip install -U ultralytics'


2024-01-30 14:48:41,671	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=1928628, ip=10.183.18.198, actor_id=3fbd9604f1420209f51e91b101000000, repr=_tune)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/tune/trainable/trainable.py", line 342, in train
    raise skipped from exception_cause(skipped)
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/air/_internal/util.py", line 88, in run
    self._ret = self._target(*self._args, **self._kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/hpc/group/borsuklab/cred/.conda/envs/yolov8_env/lib/python3.11/site-packages/ray/tune/trainable/function_trainable.py", line 115, in <lambda>
    training_func=lambda: self._trainable_func(self.config),
                          ^^^^^^^^^^^^^

In [10]:
result_grid

ResultGrid<[
  Result(
    metrics={'metrics/precision(B)': 0.8028519858278855, 'metrics/recall(B)': 0.3670430689528504, 'metrics/mAP50(B)': 0.39748359961895374, 'metrics/mAP50-95(B)': 0.2011574046913876, 'fitness': 0.22079002418414423},
    path='/work/csr33/object_detection/runs/detect/tune22/_tune_2024-01-30_12-51-34/_tune_35523_00000_0_box=0.0815,cls=2.0199,copy_paste=0.6378,degrees=16.5524,fliplr=0.0301,flipud=0.4739,hsv_h=0.0891,hsv_s=0.4417,_2024-01-30_12-51-34',
    filesystem='local',
    checkpoint=None
  )
]>

In [12]:
# trainable
# https://github.com/ultralytics/ultralytics/issues/6212

restored_tuner = tune.Tuner.restore("/work/csr33/object_detection/runs/detect/tune22/_tune_2024-01-30_12-51-34",#"/tmp/tmp_vsfwz0p/test_tune1",
                                    trainable="_tune")#/tmp/tmp_vsfwz0p/test_tune1

result_grid = restored_tuner.get_results()

# result_grid.get_best_result

In [ ]:
experiment_path = f"{storage_path}/{exp_name}"
print(f"Loading results from {experiment_path}...")



In [15]:
if result_grid.errors:
    print("One or more trials failed!")
else:
    print("No errors!")

AttributeError: 'NoneType' object has no attribute 'errors'

In [ ]:
#https://docs.ultralytics.com/reference/utils/tuner/
#https://docs.ultralytics.com/reference/utils/tuner/#ultralytics.utils.tuner.run_ray_tune


def tune(args):
    experiment = Experiment(
    api_key='CwpWEkJDRJc0rY57WYgoDuJvp',
    project_name='yolov8-ast-tume',)
    # Load the model.
    # Initialize the YOLO model

    # Log the best hyperparameters to Comet ML
    experiment.log_parameters(best_params)
    # Train the model with the best hyperparameters
    model.train()
    # Log the model to Comet ML
    experiment.log_model("YOLOv8", "yolov8.pth")


In [ ]:
if __name__ == '__main__':
    # Get the arguments
    args = get_args_parse()
    print(args)
    tune(args)